### Plot tree with matrix/heatmap

In [5]:
import toytree
import numpy as np
import toyplot

# load tree with variable name lengths
tree = toytree.tree("https://eaton-lab.org/data/Cyathophora.tre")
tree = tree.root(13) #13th node corresponds to przwilenskii outgroup

#### Method 1: 
The simplest method is to plot the tree and markers on shared coordinate axes. To make it easy to space items on the x-axis I set the tree to be 2X the width of the data (matrix), which allows me to use units of x=1 to space items on the x-axis. Then I generate a canvas and axes by drawing a tree, as usual, and here I add the data as square scatterplot markers with different opacities to represent the (randomly generated) data. 

The only tricky thing here is that you need to use `tip_labels_style` to offset the x-location of the tre tip labels, and also to extend the x-axis max domain if the names are long to prevent them from getting cut off. 

In [6]:
# generate some random data for this columns
spdata = np.random.randint(low=1, high=10, size=(tree.ntips, 5))
spdata

array([[2, 5, 7, 7, 7],
       [6, 2, 2, 4, 2],
       [5, 3, 9, 3, 5],
       [8, 6, 5, 9, 6],
       [5, 6, 7, 2, 4],
       [8, 2, 9, 4, 3],
       [4, 2, 2, 3, 2],
       [9, 2, 1, 4, 8],
       [6, 6, 1, 5, 3],
       [3, 5, 2, 8, 2],
       [7, 9, 1, 5, 1],
       [1, 4, 1, 8, 7],
       [9, 8, 7, 6, 4]])

In [11]:
# scale tree
ctree = tree.mod.edges_scale_to_root_height()

# get canvas and axes with tree plot
canvas, axes, mark = ctree.draw(
    width=600,
    height=300,
    tip_labels_align=True,
    tip_labels_style={"-toyplot-anchor-shift": "80px"}
    
);

# add n columns of data (here random data)
ncols = 5
xoffset = 1
for col in range(5):
    
    # select the column of data
    data = spdata[:, col]
    
    # plot the data column
    axes.scatterplot(
        np.repeat(col, tree.ntips) + xoffset, 
        np.arange(tree.ntips),
        marker='s',
        size=10,
        color="magenta",           
        opacity=0.1 + data[::-1] / data.max(),
        title=data,
    );

# stretch domain to fit long tip names
axes.x.domain.max = 40

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="300.0px" viewBox="0 0 600.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t8255719141974f14aede4403a937e2db"> 32082_przewalskii 33588_przewalskii 33413_thamno 30556_thamno 40578_rex 35855_rex 35236_rex 39618_rex 38362_rex 29154_superba 30686_cyathophylla 41954_cyathophylloides 41478_cyathophylloides 2 6 5 8 5 8 4 9 6 3 7 1 9 5 2 3 6 6 2 2 2 6 5 9 4 8 7 2 9 5 7 9 2 1 1 2 1 1 7 7 4 3 9 2 4 3 4 5 8 5 8 6 7 2 5 6 4 3 2 8 3 2 1 7 4

#### Method 2:

Using both a *matrix* and *cartesian* axes in *toyplot*. The key to aligning the two is that matrices have a margin of 50px by default. There aren't as many options to style matrix cells as there are in the option above. Here I used the right-side matrix labels to add and align tip names. 

In [8]:
# a random rectangular matrix
matrix = np.arange(tree.ntips * 5).reshape(tree.ntips, 5)
matrix.shape

(13, 5)

In [9]:
# create a canvas
canvas = toyplot.Canvas(width=500, height=350);

# add tree 
axes = canvas.cartesian(bounds=(50, 150, 70, 250))
tree.draw(axes=axes, tip_labels=False, tip_labels_align=True)

# add matrix
table = canvas.table(
    rows=13,
    columns=5, 
    margin=0,
    bounds=(175, 250, 65, 255),
)

colormap = toyplot.color.brewer.map("BlueRed")

# apply a color to each cell in the table
for ridx in range(matrix.shape[0]):
    for cidx in range(matrix.shape[1]):
        cell = table.cells.cell[ridx, cidx]
        cell.style = {
            "fill": colormap.colors(matrix[ridx, cidx], 0, 100), 
        }

# style the gaps between cells
table.body.gaps.columns[:] = 3
table.body.gaps.rows[:] = 3 

# hide axes coordinates
axes.show = False

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="350.0px" viewBox="0 0 500.0 350.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t122cffa5831a444999c8efde70a2bf73">

In [10]:
# create a canvas
canvas = toyplot.Canvas(width=500, height=350);

# add tree 
axes = canvas.cartesian(bounds=(50, 150, 70, 250))
tree.draw(axes=axes, tip_labels=False, tip_labels_align=True)

# add matrix
colormap = toyplot.color.brewer.map("BlueRed")
table = canvas.matrix(
    (matrix, colormap),
    bounds=(120, 300, 25, 295), 
    tshow=True,
    tlabel="Traits",
    lshow=False,
    rshow=True,
    margin=0,
    rlocator=toyplot.locator.Explicit(range(tree.ntips), tree.get_tip_labels()[::-1])
)

# hide axes coordinates
axes.show = False

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="350.0px" viewBox="0 0 500.0 350.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t484b7e50c56d4aac8ec14e8e1e08f88e"> 0 1 2 3 4 0.000000 1.000000 2.000000 3.000000 4.000000 41478_cyathophylloides 5.000000 6.000000 7.000000 8.000000 9.000000 41954_cyathophylloides 10.000000 11.000000 12.000000 13.000000 14.000000 30686_cyathophylla 15.000000 16.000000 17.000000 18.000000 19.000000 29154_superba 20.000000 21.000000 22.000000 23.000000 24.000000 38362_rex 25.000000 26.000000 27.000000 28.000000 29.000000 39618_rex 30.000000 31.000000 32.000000 33.000000 34.000000 35236_rex 35.000000 36.000000 37.000000 38.000000 39.000000 35855_rex 40.000000 41.000000 42.000000 43.000000 44.000000 40578_rex 45.000000 46.000000 47.000000 48.000000 49.000000 30556_thamno 50.000000 51.000000 52.000000 53.000000 54.000000 33413_thamno 55.000000 56.000000 57.000000 58.000000 59.000000 33588_przewalskii 60.000000 61.000000 62.000000 63.000000 64.000000 32082_przewalskii Traits